# Тестовое задание для DominiGames

### Описание задания:

1. файл test.csv 	представляет собой 	статистику продаж в сторе Amazon с 01-01-20 по 28.03.20.
2. Найти количество установок, покупок и суммарный доход (в USD) каждой игры за период с 15.01.20 по 15.02.20. Сформированный отчет должен быть представлен в одной таблице.
3. Для конвертации валют использовать API https://openexchangerates.org/ (при регистрации доступен бесплатный план на 1000 запросов в месяц)
4. Творческое задание. Придумать какую еще статистику и метрики мы можем получить из этих данных. Визуализировать.


In [4]:
import pandas as pd

In [93]:
data = pd.read_csv('testdata_DominiGames.csv')

### Краткое описание столбцов:
1. Transaction Time — дата, время и часовой пояс транзакции (в дальнейшем временем и часовым поясом пренебречь)
2. Vendor SKU — уникальный id игры
3. Title — не уникальное название игры
4. Item Name — имя события магазина
5. Item Type — тип события в магазине (Application - установка, In-App - покупка)
6. Units — количество единиц товара 		
7. Marketplace Currency — валюта, в который была совершена сделка
8. Sales Price (Marketplace Currency) — цена сделки в валюте операции

In [94]:
data

,Transaction Time,Vendor SKU,Title,Item Name,Item Type,Units,Marketplace Currency,Sales Price (Marketplace Currency)
0,2020-01-31 23:57:21 PST,com.dominigames.mt9,Hidden Object - Mystery Tales: The Other Side,Hidden Object - Mystery Tales: The Other Side,Application,1,USD,0.00
1,2020-01-31 23:55:07 GMT,com.dominigames.christmas2,The Christmas Spirit: Mother Goose's Untold Tales,The Christmas Spirit: Mother Goose's Untold Tales,Application,1,GBP,0.00
2,2020-01-31 23:54:21 GMT,com.dominigames.christmas2,The Christmas Spirit: Mother Goose's Untold Tales,The Christmas Spirit: Mother Goose's Untold Tales,Application,1,GBP,0.00
3,2020-01-31 23:52:15 GMT,com.dominigames.sl2.unlock,Hidden Object - Spirit Legends: Solar Eclipse ...,Unlock,In-App,1,GBP,1.58
4,2020-01-31 23:51:01 PST,com.dominigames.dr8.unlock,Dark Romance: Winter Lily Collector's Edition,Unlock,In-App,1,USD,2.99
...,...,...,...,...,...,...,...,...
79040,2020-03-01 00:39:34 GMT,com.dominigames.mt9,Hidden Object - Mystery Tales: The Other Side,Hidden Object - Mystery Tales: The Other Side,Application,1,GBP,0.00
79041,2020-03-01 00:21:03 PST,com.dominigames.mysterytales4,Mystery Tales: Her Own Eyes Collector's Edition,Mystery Tales: Her Own Eyes Collector's Edition,Application,1,USD,0.00
79042,2020-03-01 00:20:27 GMT,com.dominigames.sc2,Hidden Objects - Secret City: The Sunken Kingd...,Hidden Objects - Secret City: The Sunken Kingd...,Application,1,GBP,0.00
79043,2020-03-01 00:17:12 GMT,com.dominigames.mt5,Mystery Tales: Eye of the Fire Collector's Edi...,Mystery Tales: Eye of the Fire Collector's Edi...,Application,1,GBP,0.00


In [95]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79045 entries, 0 to 79044
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Transaction Time                    79045 non-null  object 
 1   Vendor SKU                          79017 non-null  object 
 2   Title                               79017 non-null  object 
 3   Item Name                           79017 non-null  object 
 4   Item Type                           79017 non-null  object 
 5   Units                               79045 non-null  int64  
 6   Marketplace Currency                79045 non-null  object 
 7   Sales Price (Marketplace Currency)  79045 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 4.8+ MB


### Выделение даты и временной зоны:

In [111]:
undetected_timezone_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'PST' not in Transaction_Time:
        if 'GMT' not in Transaction_Time:
            if 'CET' not in Transaction_Time:
                if 'CEST' not in Transaction_Time:
                    if 'EST' not in Transaction_Time:
                        if 'JST' not in Transaction_Time:
                            if 'AEDT' not in Transaction_Time:
                                if 'PDT' not in Transaction_Time:
                                    if 'BST' not in Transaction_Time:
                                        if 'EDT' not in Transaction_Time:
                                            undetected_timezone_index.append(index)
#undetected_timezone_data = data.loc[undetected_timezone_index].copy()
#undetected_timezone_data
print('Индексы с неопределёнными поясами:',undetected_timezone_index)

# Все часовые пояса:
# PST, GMT, CET, CEST, EST, JST, AEDT, PDT, BST, EDT

Индексы с неопределёнными поясами: []


In [139]:
# PST
PST_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'PST' in Transaction_Time:
        PST_index.append(index)
PST_data = data.loc[PST_index].copy()
PST_data['local_time'] = pd.to_datetime(PST_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S PST')
PST_data.loc[PST_index,'Timezone'] = ['PST']
#PST_data

In [123]:
# GMT
GMT_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'GMT' in Transaction_Time:
        GMT_index.append(index)
GMT_data = data.loc[GMT_index].copy()
GMT_data['local_time'] = pd.to_datetime(GMT_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S GMT')
GMT_data.loc[GMT_index,'Timezone'] = ['GTM']
#GMT_data

In [124]:
# CET
CET_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'CET' in Transaction_Time:
        CET_index.append(index)
CET_data = data.loc[CET_index].copy()
CET_data['local_time'] = pd.to_datetime(CET_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S CET')
CET_data.loc[CET_index,'Timezone'] = ['CET']
#CET_data

In [125]:
# CEST
CEST_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'CEST' in Transaction_Time:
        CEST_index.append(index)
CEST_data = data.loc[CEST_index].copy()
CEST_data['local_time'] = pd.to_datetime(CEST_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S CEST')
CEST_data.loc[CEST_index,'Timezone'] = ['CEST']
#EST_data

In [126]:
# EST
EST_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'CEST' not in Transaction_Time:
        if 'EST' in Transaction_Time:
            EST_index.append(index)
EST_data = data.loc[EST_index].copy()
EST_data['local_time'] = pd.to_datetime(EST_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S EST')
EST_data.loc[EST_index,'Timezone'] = ['EST']
#EST_data

In [127]:
# JST
JST_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'JST' in Transaction_Time:
        JST_index.append(index)
JST_data = data.loc[JST_index].copy()
JST_data['local_time'] = pd.to_datetime(JST_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S JST')
JST_data.loc[JST_index,'Timezone'] = ['JST']
#JST_data

In [128]:
# AEDT
AEDT_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'AEDT' in Transaction_Time:
        AEDT_index.append(index)
AEDT_data = data.loc[AEDT_index].copy()
AEDT_data['local_time'] = pd.to_datetime(AEDT_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S AEDT')
AEDT_data.loc[AEDT_index,'Timezone'] = ['AEDT']
#AEDT_data

In [129]:
# PDT
PDT_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'PDT' in Transaction_Time:
        PDT_index.append(index)
PDT_data = data.loc[PDT_index].copy()
PDT_data['local_time'] = pd.to_datetime(PDT_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S PDT')
PDT_data.loc[PDT_index,'Timezone'] = ['PDT']
#PDT_data

In [130]:
# BST
BST_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'BST' in Transaction_Time:
        BST_index.append(index)
BST_data = data.loc[BST_index].copy()
BST_data['local_time'] = pd.to_datetime(BST_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S BST')
BST_data.loc[BST_index,'Timezone'] = ['BST']
#BST_data

In [132]:
# EDT
EDT_index = []
for index in data.index:
    Transaction_Time = data.loc[index,'Transaction Time']
    if 'AEDT' not in Transaction_Time:
        if 'EDT' in Transaction_Time:
            EDT_index.append(index)
EDT_data = data.loc[EDT_index].copy()
EDT_data['local_time'] = pd.to_datetime(EDT_data['Transaction Time'], format='%Y-%m-%d %H:%M:%S EDT')
EDT_data.loc[EDT_index,'Timezone'] = ['EDT']
#EDT_data

In [142]:
# Все часовые пояса:
total_data = [PST_data, GMT_data, CET_data, CEST_data, EST_data, JST_data, AEDT_data, PDT_data, BST_data, EDT_data]
total_len = 0
for element in total_data:
    total_len = total_len + len(element)
if len(data) == total_len:
    print('Выделение верное')

Выделение верное


In [225]:
total_dataset = pd.concat([PST_data] + [GMT_data] + [CET_data] + [CEST_data] + [EST_data] + [JST_data] + [AEDT_data] + [PDT_data] + [BST_data] + [EDT_data])
#total_dataset

# Получение данных для формирования вывода:

In [226]:
# период с 15.01.20 по 15.02.20:
sellect_dataset = total_dataset.loc[(total_dataset['local_time']>='2020-01-15')&(total_dataset['local_time']<='2020-02-15')]
#sellect_dataset

In [228]:
index_dontUSD = sellect_dataset.loc[sellect_dataset['Marketplace Currency'] != 'USD'].index

index_dontzero_sales = sellect_dataset.loc[sellect_dataset['Sales Price (Marketplace Currency)'] != 0].index

sellect_data_dontUSD_sales = sellect_dataset.loc[index_dontUSD&index_dontzero_sales]

len(sellect_data_dontUSD_sales)

#sellect_data_dontUSD_sales

1641

### Суммарный доход в USD:

In [210]:
total_exchange_rate_dollar = []
api = '67cead0ca6c34fd0b9a7501092dd432e'
k = 0
for index in sellect_data_dontUSD_sales.index:
    Marketplace_Currency = sellect_data_dontUSD_sales.loc[index,'Marketplace Currency']
    local_time = sellect_data_dontUSD_sales.loc[index,'local_time']
    month = local_time.month
    if month < 10:
        month = str(0)+ str(month)
    try: 
        if month >= 10:
            month = str(month)
    except: n = 0
    day = local_time.day
    if day < 10:
        day = str(0)+ str(day)
    try: 
        if day >= 10:
            day = str(day)
    except: n = 0
    historical = str(local_time.year)+'-'+ month + '-' + day
    df = pd.read_json('https://openexchangerates.org/api/historical/'+historical+'.json?app_id='+api+'&base=USD')
    exchange_rate_dollar = df.loc[Marketplace_Currency,'rates']
    total_exchange_rate_dollar.append(exchange_rate_dollar)
    k = k + 1
    print(k,index,historical, Marketplace_Currency,exchange_rate_dollar)

1 3 2020-01-31 GBP 0.757312
2 9 2020-01-31 GBP 0.757312
3 10 2020-01-31 GBP 0.757312
4 14 2020-01-31 GBP 0.757312
5 36 2020-01-31 GBP 0.757312
6 37 2020-01-31 GBP 0.757312
7 55 2020-01-31 GBP 0.757312
8 57 2020-01-31 GBP 0.757312
9 59 2020-01-31 GBP 0.757312
10 63 2020-01-31 GBP 0.757312
11 65 2020-01-31 GBP 0.757312
12 72 2020-01-31 GBP 0.757312
13 80 2020-01-31 GBP 0.757312
14 83 2020-01-31 GBP 0.757312
15 85 2020-01-31 GBP 0.757312
16 94 2020-01-31 GBP 0.757312
17 100 2020-01-31 GBP 0.757312
18 103 2020-01-31 GBP 0.757312
19 105 2020-01-31 GBP 0.757312
20 114 2020-01-31 GBP 0.757312
21 131 2020-01-31 GBP 0.757312
22 147 2020-01-31 GBP 0.757312
23 153 2020-01-31 GBP 0.757312
24 164 2020-01-31 GBP 0.757312
25 168 2020-01-31 GBP 0.757312
26 195 2020-01-31 GBP 0.757312
27 204 2020-01-31 GBP 0.757312
28 240 2020-01-31 GBP 0.757312
29 292 2020-01-31 GBP 0.757312
30 297 2020-01-31 GBP 0.757312
31 301 2020-01-31 GBP 0.757312
32 329 2020-01-31 GBP 0.757312
33 332 2020-01-31 GBP 0.757312
34 3

236 3439 2020-01-27 GBP 0.765744
237 3443 2020-01-27 GBP 0.765744
238 3447 2020-01-27 GBP 0.765744
239 3459 2020-01-27 GBP 0.765744
240 3470 2020-01-27 GBP 0.765744
241 3476 2020-01-27 GBP 0.765744
242 3519 2020-01-27 GBP 0.765744
243 3523 2020-01-27 GBP 0.765744
244 3530 2020-01-27 GBP 0.765744
245 3538 2020-01-27 GBP 0.765744
246 3607 2020-01-27 GBP 0.765744
247 3620 2020-01-27 GBP 0.765744
248 3622 2020-01-27 GBP 0.765744
249 3624 2020-01-27 GBP 0.765744
250 3629 2020-01-27 GBP 0.765744
251 3635 2020-01-27 GBP 0.765744
252 3636 2020-01-27 GBP 0.765744
253 3641 2020-01-27 GBP 0.765744
254 3647 2020-01-27 GBP 0.765744
255 3650 2020-01-26 GBP 0.7656219999999999
256 3654 2020-01-26 GBP 0.7656219999999999
257 3659 2020-01-26 GBP 0.7656219999999999
258 3672 2020-01-26 GBP 0.7656219999999999
259 3687 2020-01-26 GBP 0.7656219999999999
260 3690 2020-01-26 GBP 0.7656219999999999
261 3744 2020-01-26 GBP 0.7656219999999999
262 3747 2020-01-26 GBP 0.7656219999999999
263 3748 2020-01-26 GBP 0.765

466 7106 2020-01-22 GBP 0.7607919999999999
467 7134 2020-01-22 GBP 0.7607919999999999
468 7138 2020-01-22 GBP 0.7607919999999999
469 7151 2020-01-22 GBP 0.7607919999999999
470 7174 2020-01-22 GBP 0.7607919999999999
471 7175 2020-01-22 GBP 0.7607919999999999
472 7196 2020-01-22 GBP 0.7607919999999999
473 7202 2020-01-22 GBP 0.7607919999999999
474 7206 2020-01-22 GBP 0.7607919999999999
475 7218 2020-01-22 GBP 0.7607919999999999
476 7227 2020-01-22 GBP 0.7607919999999999
477 7239 2020-01-22 GBP 0.7607919999999999
478 7243 2020-01-22 GBP 0.7607919999999999
479 7247 2020-01-22 GBP 0.7607919999999999
480 7269 2020-01-22 GBP 0.7607919999999999
481 7318 2020-01-22 GBP 0.7607919999999999
482 7338 2020-01-22 GBP 0.7607919999999999
483 7344 2020-01-22 GBP 0.7607919999999999
484 7350 2020-01-22 GBP 0.7607919999999999
485 7362 2020-01-22 GBP 0.7607919999999999
486 7371 2020-01-22 GBP 0.7607919999999999
487 7373 2020-01-22 GBP 0.7607919999999999
488 7394 2020-01-22 GBP 0.7607919999999999
489 7401 20

700 11094 2020-01-18 GBP 0.768315
701 11119 2020-01-18 GBP 0.768315
702 11154 2020-01-18 GBP 0.768315
703 11172 2020-01-18 GBP 0.768315
704 11189 2020-01-18 GBP 0.768315
705 11217 2020-01-17 GBP 0.768315
706 11230 2020-01-17 GBP 0.768315
707 11238 2020-01-17 GBP 0.768315
708 11248 2020-01-17 GBP 0.768315
709 11258 2020-01-17 GBP 0.768315
710 11260 2020-01-17 GBP 0.768315
711 11281 2020-01-17 GBP 0.768315
712 11312 2020-01-17 GBP 0.768315
713 11344 2020-01-17 GBP 0.768315
714 11349 2020-01-17 GBP 0.768315
715 11393 2020-01-17 GBP 0.768315
716 11403 2020-01-17 GBP 0.768315
717 11422 2020-01-17 GBP 0.768315
718 11465 2020-01-17 GBP 0.768315
719 11473 2020-01-17 GBP 0.768315
720 11582 2020-01-17 GBP 0.768315
721 11587 2020-01-17 GBP 0.768315
722 11679 2020-01-17 GBP 0.768315
723 11686 2020-01-17 GBP 0.768315
724 11722 2020-01-17 GBP 0.768315
725 11746 2020-01-17 GBP 0.768315
726 11763 2020-01-17 GBP 0.768315
727 11807 2020-01-17 GBP 0.768315
728 11856 2020-01-17 GBP 0.768315
729 11921 2020

930 41298 2020-02-12 GBP 0.7717269999999999
931 41367 2020-02-12 GBP 0.7717269999999999
932 41379 2020-02-12 GBP 0.7717269999999999
933 41405 2020-02-12 GBP 0.7717269999999999
934 41415 2020-02-12 GBP 0.7717269999999999
935 41431 2020-02-12 GBP 0.7717269999999999
936 41432 2020-02-12 GBP 0.7717269999999999
937 41443 2020-02-12 GBP 0.7717269999999999
938 41451 2020-02-11 GBP 0.7717
939 41457 2020-02-11 GBP 0.7717
940 41458 2020-02-11 GBP 0.7717
941 41563 2020-02-11 GBP 0.7717
942 41608 2020-02-11 GBP 0.7717
943 41626 2020-02-11 GBP 0.7717
944 41636 2020-02-11 GBP 0.7717
945 41679 2020-02-11 GBP 0.7717
946 41686 2020-02-11 GBP 0.7717
947 41701 2020-02-11 GBP 0.7717
948 41711 2020-02-11 GBP 0.7717
949 41712 2020-02-11 GBP 0.7717
950 41768 2020-02-11 GBP 0.7717
951 41790 2020-02-11 GBP 0.7717
952 41817 2020-02-11 GBP 0.7717
953 41875 2020-02-11 GBP 0.7717
954 41939 2020-02-11 GBP 0.7717
955 41943 2020-02-11 GBP 0.7717
956 41975 2020-02-11 GBP 0.7717
957 42008 2020-02-11 GBP 0.7717
958 4202

1140 45559 2020-02-07 GBP 0.775976
1141 45576 2020-02-07 GBP 0.775976
1142 45641 2020-02-07 GBP 0.775976
1143 45662 2020-02-07 GBP 0.775976
1144 45665 2020-02-07 GBP 0.775976
1145 45718 2020-02-07 GBP 0.775976
1146 45722 2020-02-07 GBP 0.775976
1147 45764 2020-02-07 GBP 0.775976
1148 45778 2020-02-07 GBP 0.775976
1149 45787 2020-02-07 GBP 0.775976
1150 45790 2020-02-07 GBP 0.775976
1151 45796 2020-02-06 GBP 0.7733869999999999
1152 45826 2020-02-06 GBP 0.7733869999999999
1153 45839 2020-02-06 GBP 0.7733869999999999
1154 45842 2020-02-06 GBP 0.7733869999999999
1155 45862 2020-02-06 GBP 0.7733869999999999
1156 45866 2020-02-06 GBP 0.7733869999999999
1157 45875 2020-02-06 GBP 0.7733869999999999
1158 45880 2020-02-06 GBP 0.7733869999999999
1159 45881 2020-02-06 GBP 0.7733869999999999
1160 45894 2020-02-06 GBP 0.7733869999999999
1161 45925 2020-02-06 GBP 0.7733869999999999
1162 45955 2020-02-06 GBP 0.7733869999999999
1163 45975 2020-02-06 GBP 0.7733869999999999
1164 45977 2020-02-06 GBP 0.77

1352 49465 2020-02-02 GBP 0.758666
1353 49484 2020-02-02 GBP 0.758666
1354 49490 2020-02-02 GBP 0.758666
1355 49495 2020-02-02 GBP 0.758666
1356 49519 2020-02-02 GBP 0.758666
1357 49521 2020-02-02 GBP 0.758666
1358 49535 2020-02-02 GBP 0.758666
1359 49544 2020-02-02 GBP 0.758666
1360 49553 2020-02-02 GBP 0.758666
1361 49561 2020-02-02 GBP 0.758666
1362 49562 2020-02-02 GBP 0.758666
1363 49577 2020-02-02 GBP 0.758666
1364 49603 2020-02-02 GBP 0.758666
1365 49607 2020-02-02 GBP 0.758666
1366 49624 2020-02-02 GBP 0.758666
1367 49644 2020-02-02 GBP 0.758666
1368 49645 2020-02-02 GBP 0.758666
1369 49659 2020-02-02 GBP 0.758666
1370 49665 2020-02-02 GBP 0.758666
1371 49669 2020-02-02 GBP 0.758666
1372 49701 2020-02-02 GBP 0.758666
1373 49703 2020-02-02 GBP 0.758666
1374 49744 2020-02-02 GBP 0.758666
1375 49748 2020-02-02 GBP 0.758666
1376 49753 2020-02-02 GBP 0.758666
1377 49760 2020-02-02 GBP 0.758666
1378 49767 2020-02-02 GBP 0.758666
1379 49793 2020-02-02 GBP 0.758666
1380 49835 2020-02-0

1575 43053 2020-02-09 EUR 0.913296
1576 43075 2020-02-09 EUR 0.913296
1577 43148 2020-02-09 EUR 0.913296
1578 43435 2020-02-09 EUR 0.913296
1579 43451 2020-02-09 EUR 0.913296
1580 43985 2020-02-08 EUR 0.913551
1581 44321 2020-02-08 EUR 0.913551
1582 44470 2020-02-08 EUR 0.913551
1583 44497 2020-02-08 EUR 0.913551
1584 44498 2020-02-08 EUR 0.913551
1585 44592 2020-02-08 EUR 0.913551
1586 44594 2020-02-08 EUR 0.913551
1587 44920 2020-02-07 EUR 0.9136179999999999
1588 44942 2020-02-07 EUR 0.9136179999999999
1589 44988 2020-02-07 EUR 0.9136179999999999
1590 45032 2020-02-07 EUR 0.9136179999999999
1591 45036 2020-02-07 EUR 0.9136179999999999
1592 45059 2020-02-07 EUR 0.9136179999999999
1593 45237 2020-02-07 EUR 0.9136179999999999
1594 45435 2020-02-07 EUR 0.9136179999999999
1595 45610 2020-02-07 EUR 0.9136179999999999
1596 45996 2020-02-06 EUR 0.910647
1597 46018 2020-02-06 EUR 0.910647
1598 46379 2020-02-06 EUR 0.910647
1599 46589 2020-02-06 EUR 0.910647
1600 46907 2020-02-05 EUR 0.908967


In [229]:
data_for_df = []
for element in total_exchange_rate_dollar:
    data_for_df.append([element])

new_df = pd.DataFrame(data = data_for_df, columns = ['rate_dollar'])
new_df.index = sellect_data_dontUSD_sales.index

sellect_data_dontUSD_sales['rate_dollar'] = new_df['rate_dollar']
#sellect_data_dontUSD_sales

In [232]:
def add_convert_to_dollars(data):
    sales_price = data['Sales Price (Marketplace Currency)']
    rate_dollar = data['rate_dollar']
    sales_price_in_dollars = sales_price / rate_dollar
    return sales_price_in_dollars
sellect_data_dontUSD_sales['sales_price_in_dollars'] = sellect_data_dontUSD_sales.apply(add_convert_to_dollars, axis=1)
#sellect_data_dontUSD_sales

In [280]:
index_USD = sellect_dataset.loc[sellect_dataset['Marketplace Currency'] == 'USD'].index
sellect_data_USD_sales = sellect_dataset.loc[index_USD]
sellect_data_USD_sales.loc[index_USD,'sales_price_in_dollars'] = sellect_dataset['Sales Price (Marketplace Currency)']

index_zero_sales = sellect_dataset.loc[sellect_dataset['Sales Price (Marketplace Currency)'] == 0].index
sellect_data_zero_sales = sellect_dataset.loc[index_zero_sales]
sellect_data_zero_sales.loc[index_zero_sales,'sales_price_in_dollars'] = 0

final_sales_data = pd.concat([sellect_data_USD_sales] + [sellect_data_zero_sales] + [sellect_data_dontUSD_sales])

sellect_dataset = sellect_dataset.drop_duplicates()
final_sales_data = final_sales_data.drop_duplicates()

if len(sellect_dataset) == len(final_sales_data):
    print('размеры таблиц не отличаются')
    
final_sales_data = final_sales_data.drop(['Transaction Time','Title','Item Name','Units','rate_dollar'], axis='columns')

размеры таблиц не отличаются


In [299]:
final_sales_data

,Vendor SKU,Item Type,Marketplace Currency,Sales Price (Marketplace Currency),local_time,Timezone,sales_price_in_dollars
0,com.dominigames.mt9,Application,USD,0.00,2020-01-31 23:57:21,PST,0.000000
4,com.dominigames.dr8.unlock,In-App,USD,2.99,2020-01-31 23:51:01,PST,2.990000
5,com.dominigames.mt8,Application,USD,0.00,2020-01-31 23:45:08,PST,0.000000
6,com.dominigames.cc8.unlock,In-App,USD,2.99,2020-01-31 23:43:18,PST,2.990000
11,com.dominigames.cc8,Application,USD,0.00,2020-01-31 23:30:18,PST,0.000000
...,...,...,...,...,...,...,...
39844,com.dominigames.sc3.unlock,In-App,CAD,7.97,2020-02-14 10:34:38,EST,6.014663
39845,com.dominigames.mt9.unlock,In-App,CAD,7.96,2020-02-14 10:30:38,EST,6.007116
45438,com.dominigames.mt9.unlock,In-App,CAD,7.96,2020-02-07 14:27:49,EST,5.982309
49515,com.dominigames.christmas.unlock,In-App,CAD,3.98,2020-02-02 20:40:40,EST,3.005597


# Формирование выводов:

In [331]:
vivod_data = final_sales_data.groupby('Vendor SKU')['sales_price_in_dollars'].sum()
vivod_index = vivod_data.index
vivod_data = pd.DataFrame(data = vivod_data0.values, columns = ['total_sales_price_in_dollars'])
vivod_data.index = vivod_index
#vivod_data

In [352]:
data_Item_Type = []

Vendor_SKU = final_sales_data['Vendor SKU'].value_counts().index
#Vendor_SKU.index
for element in Vendor_SKU:
    sellect_game = final_sales_data.loc[final_sales_data['Vendor SKU'] == element]
    sellect_game['Item Type'].value_counts()
    sellect_game_item_type = sellect_game['Item Type'].value_counts()
    try:
        Application = sellect_game_item_type['Application']
    except:
        Application = 0
    try:
        In_App = sellect_game_item_type['In-App']
    except:
        In_App = 0
    data_Item_Type.append([element,Application,In_App])
    #
vivod_data1 = pd.DataFrame(data = data_Item_Type, columns = ['Vendor SKU','Application','In_App'])
vivod_data1.index = vivod_data1['Vendor SKU']
vivod_data1 = vivod_data1.drop(['Vendor SKU'], axis='columns')
vivod_data1['total_sales_price_in_dollars'] = vivod_data['total_sales_price_in_dollars']
# сортировка:
vivod_data1 = vivod_data1.sort_values(by = 'total_sales_price_in_dollars', ascending = False)
#vivod_data1

In [357]:
for element in vivod_data1.index:
    Application = vivod_data1.loc[element,'Application']
    if '.unlock' not in element:
        fest_index = element
        for element in vivod_data1.index:
            if '.unlock' in element and fest_index in element:
                vivod_data1.loc[element,'Application'] = vivod_data1.loc[fest_index,'Application']
#vivod_data1

In [370]:
in_app_nozero_index = vivod_data1.loc[vivod_data1['In_App']!=0].index
vivod_data2 = vivod_data1.loc[in_app_nozero_index]

revenue_dollars_per_installation_per_sellect_month = []

for element in vivod_data2.index:
    Application = vivod_data2.loc[element,'Application']
    total_sales_price_in_dollars = vivod_data2.loc[element,'total_sales_price_in_dollars']
    dollars_per_installation = total_sales_price_in_dollars / Application
    revenue_dollars_per_installation_per_sellect_month.append([dollars_per_installation])
#
data_revenue_dollars_per_installation_per_sellect_month = pd.DataFrame(data = revenue_dollars_per_installation_per_sellect_month, columns = ['revenue_dollars_per_installation_per_sellect_month'])
data_revenue_dollars_per_installation_per_sellect_month.index = vivod_data2.index

C:\Users\Stas\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


In [371]:
vivod_data2['revenue_dollars_per_installation_per_sellect_month'] = data_revenue_dollars_per_installation_per_sellect_month['revenue_dollars_per_installation_per_sellect_month']

In [372]:
vivod_data2 # revenue_dollars_per_installation_per_sellect_month - выручка в долларах на установку приложения в выбранном месяце

,Application,In_App,total_sales_price_in_dollars,revenue_dollars_per_installation_per_sellect_month
Vendor SKU,,,,
com.dominigames.mt9.unlock,3639,882,3320.467031,0.912467
com.dominigames.cc8.unlock,1907,559,1534.020160,0.804415
com.dominigames.sc3.unlock,528,271,1296.001015,2.454547
com.dominigames.sl2.unlock,1421,343,987.460313,0.694905
com.dominigames.ch3.unlock,3905,242,757.444723,0.193968
com.dominigames.hn2.unlock,834,162,548.046094,0.657130
com.dominigames.ext.mt7.unlock,941,161,520.150964,0.552764
com.dominigames.mt8.unlock,774,157,514.813803,0.665134
com.dominigames.dr8.unlock,444,118,405.945694,0.914292
